
# Домашнее задание 2. Поисковая система для документов

**Модуль 2. Классический поиск и рекуррентные архитектуры**

**ФИО студента:** Поликарпов Дмитрий Александрович

**Дата выполнения:** 27.09.2025

## Описание задания

В этом задании вы разработаете полнофункциональную поисковую систему, включающую:
1. **Предобработку корпуса.**
2. **BM25.**
3. **Векторный поиск** — на основе эмбеддингов.
4. **Гибридный поиск** — комбинация BM25 и векторного поиска.
5. **Выбор метрики и оценку качества** — для конкретной задачи.

---

## Установка и импорт библиотек

In [ ]:
import re
import math
import time
import pickle
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Set
from collections import defaultdict, Counter

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm

# NLP
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy3

# Векторный поиск
from sentence_transformers import SentenceTransformer
import faiss

# BM25
from rank_bm25 import BM25Okapi

# Создание директорий
for dir_name in ['data', 'indices', 'models', 'results', 'tests']:
    Path(dir_name).mkdir(exist_ok=True)

# Загрузка NLTK ресурсов
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

# Инициализация морфологического анализатора
morph = pymorphy3.MorphAnalyzer()


## Часть 1. Подготовка данных

1. Загрузите и изучите предложенный датасет.  
2. Реализуйте функцию предобработки текста, которая включает:
- Лемматизацию с использованием pymorphy3.
- Удаление стоп-слов и пунктуации.  
3. Обработайте весь корпус документов и сохраните результат для последующих шагов.  


In [10]:
from datasets import load_dataset

# Загружаем корпус документов
ds = load_dataset("MLNavigator/russian-retrieval")
df = pd.DataFrame(ds['train'])
questions_df = df[['text','q']]


# Уберем дубли, так как датасет имеет соответствие много вопросов -> один документ
documents = df['text'].drop_duplicates().to_list()

Generating train split: 100%|██████████| 90556/90556 [00:09<00:00, 9797.89 examples/s] 


In [14]:
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

russian_stop_words = set(stopwords.words('russian'))

def preprocess_text(text):

    if not isinstance(text, str):
        return []
    
    text = text.lower()
    tokens = word_tokenize(text, language='russian')
    
    processed_tokens = []
    for token in tokens:

        if not token.isalpha():
            continue
        
        parsed = morph.parse(token)[0]
        lemma = parsed.normal_form
        
        if lemma not in russian_stop_words and len(lemma) > 2:
            processed_tokens.append(lemma)
            
    return processed_tokens

processed_documents = []
for i, doc in enumerate(documents):
    if i % 1000 == 0:
        print(f"Обработано {i}/{len(documents)} документов")
    
    processed_doc = preprocess_text(doc)
    processed_documents.append(processed_doc)

print("\nПример предобработки:")
print("Оригинал:", documents[0][:100] + "...")
print("После обработки:", ' '.join(processed_documents[0][:10]) + "...")

with open('processed_documents.pkl', 'wb') as f:
    pickle.dump(processed_documents, f)

Обработано 0/9076 документов
Обработано 1000/9076 документов
Обработано 2000/9076 документов
Обработано 3000/9076 документов
Обработано 4000/9076 документов
Обработано 5000/9076 документов
Обработано 6000/9076 документов
Обработано 7000/9076 документов
Обработано 8000/9076 документов
Обработано 9000/9076 документов

Пример предобработки:
Оригинал: В протерозойских отложениях органические остатки встречаются намного чаще, чем в архейских. Они пред...
После обработки: протерозойский отложение органический остаток встречаться намного частый архейский представить известковый...


## Часть 2. Реализация BM25

1. Постройте инвертированный индекс для корпуса. Индекс должен содержать частоту термина в документе (TF) и документную частоту (DF).
2. Реализуйте функцию поиска BM25 с нуля. Формула для ранжирования:
score(D, Q) = Σ IDF(qi) * (f(qi, D) * (k1 + 1)) / (f(qi, D) + k1 * (1 - b + b * |D| / avgdl))
3. Проведите оптимизацию гиперпараметра k1, чтобы улучшить качество поиска.

In [ ]:
inverted_index = defaultdict(dict)
doc_lengths = {}
N = len(processed_documents)

for doc_id, tokens in enumerate(processed_documents):
    term_freqs = Counter(tokens)
    doc_lengths[doc_id] = len(tokens)
    
    for term, freq in term_freqs.items():
        inverted_index[term][doc_id] = freq

avgdl = sum(doc_lengths.values()) / N
print(f"Документов: {N}, средняя длина: {avgdl:.2f}")

Документов: 9076, средняя длина: 69.51


In [22]:
def compute_idf(term, N=N):
    df = len(inverted_index.get(term, {}))
    return math.log((N - df + 0.5) / (df + 0.5) + 1)

def bm25_score(query_tokens, doc_id, k1=1.5, b=0.75):
    score = 0.0
    doc_len = doc_lengths[doc_id]
    
    for term in query_tokens:
        if term not in inverted_index:
            continue
        
        idf = compute_idf(term)
        f = inverted_index[term].get(doc_id, 0)
        if f == 0:
            continue
        
        denom = f + k1 * (1 - b + b * doc_len / avgdl)
        score += idf * (f * (k1 + 1)) / denom
    return score

def bm25_search(query, top_k=5, k1=1.5, b=0.75):
    query_tokens = preprocess_text(query)
    scores = {}
    
    for term in query_tokens:
        if term not in inverted_index:
            continue
        for doc_id in inverted_index[term]:
            scores.setdefault(doc_id, 0)
            scores[doc_id] += bm25_score(query_tokens, doc_id, k1, b)
    
    ranked_docs = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    return [(doc_id, score, documents[doc_id][:200]) for doc_id, score in ranked_docs[:top_k]]


In [ ]:
def evaluate_k1(k1_values, sample_size=200, top_k=5):
    results = {}
    sample = questions_df.sample(sample_size, random_state=42)
    
    for k1 in k1_values:
        hits = 0
        for _, row in sample.iterrows():
            query = row['q']
            doc_text = row['text']
            
            retrieved = bm25_search(query, top_k=top_k, k1=k1)
            if any(doc_text[:100] in r[2] for r in retrieved):
                hits += 1
        acc = hits / sample_size
        results[k1] = acc
        print(f"k1={k1:.2f}: accuracy={acc:.3f}")
    
    return results

k1_values = [0.5, 1.0, 1.5, 2.0, 2.5]
results = evaluate_k1(k1_values)
best_k1 = max(results, key=results.get)
print("Лучшее k1:", best_k1)

k1=0.50: accuracy=0.940
k1=1.00: accuracy=0.945
k1=1.50: accuracy=0.945
k1=2.00: accuracy=0.950
k1=2.50: accuracy=0.950
Лучшее k1: 2.0


In [ ]:
test_queries = [
    "машинное обучение",
    "нейронные сети",
    "поисковые системы BM25",
    "Python программирование"
]

print("Тестирование BM25:")
for query in test_queries:
    results = bm25_search(query, top_k=3)
    print(f"\nЗапрос: '{query}'")
    for doc_id, score, snippet in results:
        print(f"  [{score:.3f}] {snippet}...")


Тестирование BM25:

Запрос: 'машинное обучение'
  [10.670] Особенность программного обеспечения состоит в том, что оно производится в одной форме — в виде исходного текста, а распространяется и используется часто в другой — в виде исполнимых программ, машинны...
  [10.344] Промышленный переворот, произошедший с 60-х годов XVIII до первой четверти XIX веко́в в Великобритании, вызвал переход от мануфактуры к крупной машинной индустрии сначала в самой Великобритании, а зат...
  [8.975] Цифровые обучающие игры отличаются от традиционных обучающих игр и не основанного на играх электронного обучения тем, что они используют методы мотивации развлекательных игр, чтобы достичь своих образ...

Запрос: 'нейронные сети'
  [36.560] Области мозга, постоянно используемые, когда человек занят вопросами морали, были исследованы качественными методами мета-анализа изменений мозговой активности и результаты этих исследований отражены ...
  [7.731] Двусторонние рынки (двусторонние сети) — сетевые рынки, к

## Часть 3. Векторный поиск

1. Используйте предобученную модель sentence-transformers для получения векторных представлений (эмбеддингов) всех документов.
2. Создайте индекс для быстрого поиска ближайших соседей с помощью faiss-cpu.
3. Реализуйте функцию векторного поиска, которая по запросу находит top-k наиболее близких документов.


In [30]:
model = SentenceTransformer("BAAI/bge-m3", device="cpu")

doc_embeddings = model.encode(documents, convert_to_numpy=True, show_progress_bar=True)

print("Эмбеддинги документов готовы:", doc_embeddings.shape)

Batches: 100%|██████████| 284/284 [37:03<00:00,  7.83s/it]

Эмбеддинги документов готовы: (9076, 1024)


In [31]:
d = doc_embeddings.shape[1]

index = faiss.IndexFlatIP(d)  
faiss.normalize_L2(doc_embeddings)  
index.add(doc_embeddings)

print("Индекс построен. В нём документов:", index.ntotal)

Индекс построен. В нём документов: 9076


In [ ]:
def vector_search(query, top_k=5):
    query_embedding = model.encode([query], convert_to_numpy=True)
    faiss.normalize_L2(query_embedding)

    D, I = index.search(query_embedding, top_k) 
    
    results = []
    for score, doc_id in zip(D[0], I[0]):
        results.append((doc_id, float(score), documents[doc_id][:200]))
    return results

In [33]:
print("Тестирование векторного поиска:")
for query in test_queries:
    results = vector_search(query, top_k=3)
    print(f"\nЗапрос: '{query}'")
    for doc_id, score, snippet in results:
        print(f"  [{score:.3f}] {snippet}...")


Тестирование векторного поиска:

Запрос: 'машинное обучение'
  [0.484] Моделирование различных процессов, например в гидродинамике, физике, электрике, электронных системах и цепях, а также для моделирования общества и социальных ситуаций (в частности, военных игр), учиты...
  [0.483] Принцип Парето лежит в основании идеи компьютерных RISC-процессоров (впрочем, неизвестно, опирались ли авторы идеи на известный им принцип или повторно изобрели его сами). В то время как электронная п...
  [0.482] По словам Питера Деннинга, к фундаментальным вопросам информатики относится следующий вопрос: Что может быть эффективно автоматизировано? Изучение теории алгоритмов сфокусировано на поиске ответов на ...

Запрос: 'нейронные сети'
  [0.524] Области мозга, постоянно используемые, когда человек занят вопросами морали, были исследованы качественными методами мета-анализа изменений мозговой активности и результаты этих исследований отражены ...
  [0.503] Нервная ткань млекопитающих, как и у других поз

## Часть 4. Гибридный поиск

1. Разработайте функцию, которая комбинирует результаты ранжирования от BM25 и векторного поиска.
2. Реализуйте механизм взвешивания скоров с помощью параметра α:
hybrid_score = α * bm25_score + (1 - α) * vector_score
3. Проведите автоматическую оптимизацию параметра α на валидационном наборе данных.


In [ ]:
def hybrid_search(query, top_k=5, alpha=0.5, k_bm25=10, k_vec=10):
    bm25_results = bm25_search(query, top_k=k_bm25)
    bm25_scores = {doc_id: score for doc_id, score, _ in bm25_results}
    
    vec_results = vector_search(query, top_k=k_vec)
    vec_scores = {doc_id: score for doc_id, score, _ in vec_results}
    
    all_doc_ids = set(bm25_scores.keys()) | set(vec_scores.keys())
    hybrid_scores = {}
    
    for doc_id in all_doc_ids:
        bm25_score = bm25_scores.get(doc_id, 0.0)
        vec_score = vec_scores.get(doc_id, 0.0)
        hybrid_scores[doc_id] = alpha * bm25_score + (1 - alpha) * vec_score
    
    ranked = sorted(hybrid_scores.items(), key=lambda x: x[1], reverse=True)
    return [(doc_id, score, documents[doc_id][:200]) for doc_id, score in ranked[:top_k]]

In [35]:
# Тестируем гибридный поиск
print("Тестирование гибридного поиска:")
for query in test_queries:
    results = hybrid_search(query, top_k=3, alpha=0.6)
    print(f"\nЗапрос: '{query}'")
    for doc_id, score, snippet in results:
        print(f"  [{score:.3f}] {snippet}...")

Тестирование гибридного поиска:

Запрос: 'машинное обучение'
  [6.402] Особенность программного обеспечения состоит в том, что оно производится в одной форме — в виде исходного текста, а распространяется и используется часто в другой — в виде исполнимых программ, машинны...
  [6.207] Промышленный переворот, произошедший с 60-х годов XVIII до первой четверти XIX веко́в в Великобритании, вызвал переход от мануфактуры к крупной машинной индустрии сначала в самой Великобритании, а зат...
  [5.385] Цифровые обучающие игры отличаются от традиционных обучающих игр и не основанного на играх электронного обучения тем, что они используют методы мотивации развлекательных игр, чтобы достичь своих образ...

Запрос: 'нейронные сети'
  [22.146] Области мозга, постоянно используемые, когда человек занят вопросами морали, были исследованы качественными методами мета-анализа изменений мозговой активности и результаты этих исследований отражены ...
  [4.638] Двусторонние рынки (двусторонние сети) — сетев

## Часть 5. Оценка качества

1. Выберите и **обоснуйте метрику** для оценки качества вашей поисковой системы (например, MRR, MAP@k или NDCG@k). **Обязательно подумайте о том, какой топ-к нужно выбрать исходя из данных**.
2. **Создайте небольшой датасет для оценки**, состоящий из запросов и релевантных им документов.  
3. **Сравните качество** всех трех реализованных подходов (BM25, векторный, гибридный) на вашем датасете.  


In [ ]:
eval_sample = questions_df.sample(200, random_state=42).reset_index(drop=True)
eval_dataset = list(zip(eval_sample["q"], eval_sample["text"]))

In [ ]:
def mrr_at_k(search_fn, eval_dataset, k=5):
    reciprocal_ranks = []
    
    for query, gold_doc in eval_dataset:
        results = search_fn(query, top_k=k)
        
        rank = 0
        for i, (doc_id, score, snippet) in enumerate(results, start=1):
            if gold_doc[:100] in snippet:
                rank = i
                break
        
        if rank > 0:
            reciprocal_ranks.append(1.0 / rank)
        else:
            reciprocal_ranks.append(0.0)
    
    return sum(reciprocal_ranks) / len(reciprocal_ranks)

In [38]:
print("Оценка качества моделей (MRR@5):")

mrr_bm25 = mrr_at_k(bm25_search, eval_dataset, k=5)
print(f"BM25: {mrr_bm25:.3f}")

mrr_vector = mrr_at_k(vector_search, eval_dataset, k=5)
print(f"Vector: {mrr_vector:.3f}")

mrr_hybrid = mrr_at_k(lambda q, top_k: hybrid_search(q, top_k=top_k, alpha=0.6),
                      eval_dataset, k=5)
print(f"Hybrid: {mrr_hybrid:.3f}")

Оценка качества моделей (MRR@5):
BM25: 0.892
Vector: 0.828
Hybrid: 0.894


### Выводы:
- Для оценки выбрана `MRR@5`, т.к. у нас один релевантный документ на запрос, и важно оценить не только факт нахождения, но и позицию: чем выше документ, тем больше вклад в метрику.
- Топ-5 достаточно широк, чтобы система имела шанс поймать релевантный документ, даже если он не на первой позиции.
- `BM25` будет сильнее на точных совпадениях (например, технические термины).
- `Vector search` лучше на семантических запросах, но требует времени на создание векторной БД.
- `Hybrid` почти всегда обгонит обе модели, потому что комбинирует плюсы.